# Optical Flow

In [2]:
import numpy as np
import cv2
import time

## TODO 1:
Finish the following code

In [3]:
# Parameters for Lucas-Kanade optical flow
lk_params = dict( winSize = (15, 15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
#TO-DO Read up on Lucas-Kanade Optical Flow. What parameters need to be set? Hint: This should be made as a dictionary.

# Parameters for corner detection using Shi-Tomasi method
feature_params = dict( maxCorners = 50, qualityLevel = 0.01, minDistance = 10, blockSize = 10)
#TO-DO Read up on Shi-Tomasi method. What parameters need to be set? Hint: This should be made as a dictionary.

In [4]:
# Variables for tracking
trajectory_len = 40      # Length of the trajectory
detect_interval = 5      # Interval for detecting new features
trajectories = []        # List to store trajectories
frame_idx = 0            # Frame index

## TODO 2:
Finish the following code

In [7]:
# Capture video from the first camera device
cap = cv2.VideoCapture('testCV.mp4')
p = None

while True:
    #Save the value of p from the previous iteration
    if p != None:
        old_p = p

    # Store the start time to calculate FPS
    start = time.time

    # Read a new frame from the video capture
    suc, frame = cap.read()
    if not suc:
        break  # Break if there is an issue reading the frame
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale
    img = frame.copy()  # Create a copy of the frame for displaying results

    # Calculate optical flow for the existing trajectories
    if len(trajectories) > 0:
        img0, img1 = prev_gray, frame_gray
        #TO-DO
        po, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, old_p, None, **lk_params)
        # Get the last points of trajectories
        # Calculate forward optical flow
        # Calculate backward optical flow
        # Calculate difference between forward-backward flow
        # Identify good points with small differences

        new_trajectories = []

        # TO-DO
        # Update the trajectories with the new points and draw the new points on the image
        # Make sure to trim down the trajectories when they get too long

        trajectories = new_trajectories

        # Draw the trajectories on the image
        cv2.polylines(img, [np.int32(trajectory) for trajectory in trajectories], False, (0, 255, 0))
        cv2.putText(img, 'track count: %d' % len(trajectories), (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)

    # Detect new features at specified intervals
    if frame_idx % detect_interval == 0:
        mask = np.zeros_like(frame_gray)  # Create an empty mask
        mask[:] = 255  # Set all pixels to 255 (white)

        # Mask out regions where features are already tracked
        for x, y in [np.int32(trajectory[-1]) for trajectory in trajectories]:
            cv2.circle(mask, (x, y), 5, 0, -1)  # Draw circles on the mask

        # Detect good features to track
        p = cv2.goodFeaturesToTrack(frame_gray, mask = mask, **feature_params)
        if p is not None:
            # Add new features to the trajectories
            for x, y in np.float32(p).reshape(-1, 2):
                trajectories.append([(x, y)])

    frame_idx += 1  # Increment frame index
    prev_gray = frame_gray  # Update previous frame

    # End time to calculate FPS
    end = time.time()
    # Calculate FPS
    fps = 1 / (end - start)
    
    # Display FPS on the image
    cv2.putText(img, f"{fps:.2f} FPS", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # Show the image with optical flow
    cv2.imshow('Optical Flow', img)
    # Show the mask
    cv2.imshow('Mask', mask)

    # Exit on 'q' key press
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()


TypeError: unsupported operand type(s) for -: 'float' and 'builtin_function_or_method'

In [6]:
import numpy as np
import cv2
import time


lk_params = dict(winSize  = (15, 15),
                maxLevel = 2,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

feature_params = dict(maxCorners = 100,
                    qualityLevel = 0.3,
                    minDistance = 10,
                    blockSize = 7 )


trajectory_len = 40
detect_interval = 5
trajectories = []
frame_idx = 0


cap = cv2.VideoCapture('testCV.mp4')

while True:

    # start time to calculate FPS
    start = time.time()

    suc, frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img = frame.copy()

    # Calculate optical flow for a sparse feature set using the iterative Lucas-Kanade Method
    if len(trajectories) > 0:
        img0, img1 = prev_gray, frame_gray
        p0 = np.float32([trajectory[-1] for trajectory in trajectories]).reshape(-1, 1, 2)
        p1, _st, _err = cv2.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)
        p0r, _st, _err = cv2.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)
        d = abs(p0-p0r).reshape(-1, 2).max(-1)
        good = d < 1

        new_trajectories = []

        # Get all the trajectories
        for trajectory, (x, y), good_flag in zip(trajectories, p1.reshape(-1, 2), good):
            if not good_flag:
                continue
            trajectory.append((x, y))
            if len(trajectory) > trajectory_len:
                del trajectory[0]
            new_trajectories.append(trajectory)
            # Newest detected point
            cv2.circle(img, (int(x), int(y)), 2, (0, 0, 255), -1)

        trajectories = new_trajectories

        # Draw all the trajectories
        cv2.polylines(img, [np.int32(trajectory) for trajectory in trajectories], False, (0, 255, 0))
        cv2.putText(img, 'track count: %d' % len(trajectories), (20, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0,255,0), 2)


    # Update interval - When to update and detect new features
    if frame_idx % detect_interval == 0:
        mask = np.zeros_like(frame_gray)
        mask[:] = 255

        # Lastest point in latest trajectory
        for x, y in [np.int32(trajectory[-1]) for trajectory in trajectories]:
            cv2.circle(mask, (x, y), 5, 0, -1)

        # Detect the good features to track
        p = cv2.goodFeaturesToTrack(frame_gray, mask = mask, **feature_params)
        if p is not None:
            # If good features can be tracked - add that to the trajectories
            for x, y in np.float32(p).reshape(-1, 2):
                trajectories.append([(x, y)])


    frame_idx += 1
    prev_gray = frame_gray

    # End time
    end = time.time()
    # calculate the FPS for current frame detection
    fps = 1 / (end-start)
    
    # Show Results
    cv2.putText(img, f"{fps:.2f} FPS", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Optical Flow', img)
    cv2.imshow('Mask', mask)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.5.4) ./modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
